# DATA CLEANING AND TRANSFORMATION

In [1]:
# libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

import re

from tqdm import tqdm
import time

import sqlite3

import os
import random
import string

In [2]:
# PANDAS OPTIONS
# Set maximum number of columns and rows to display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Set the maximum column width to a high value
pd.set_option('display.max_colwidth', 1000)

In [3]:
# read back raw data from csv
df = pd.read_csv('scraped_data.csv')

df.head(2)

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description
0,BOITE DE 325 CARTOUCHES 22LR FEDERAL GOLD PLATED HOLLOW POINT 36 GRAINS,https://www.naturabuy.fr/BOITE-DE-325-CARTOUCHES-22LR-FEDERAL-GOLD-PLATED-HOLLOW-POINT-36-GRAINS-item-10619103.html,Federal,Neuf,"46,90 €","12,00 €","Partager\n\n\n Vendre le même objet \nSignaler cet objet\n\n \nDescription produit\n\n\nMunitions tir de loisir, 22LR, Plombs 4,5mm et 9mm Flobert > Munitions - Balles 22LR\n\n\nMarque : FederalEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Creuse \n\n\n\n\n\nboite de 325 cartouches de marque Federal 22lr 36 grains round nose hollow point cuivree\nexcellente cartouche pour le tir dans les armes a verrou comme semi automatique\n \n marque : Fédéral\nréférence : 725\nType d’ogive round nose cuivree a pointe creuse\nvitesse 384 mètres par secondes\npoids ogive : 36 grains\n \nen stock envoi des reception du paiement\n \nvente uniquement aux personnes ayant une licence de tir ou permis de chasse avec validation"
1,Boîte ancienne 22 LR Remington 22 Target,https://www.naturabuy.fr/Boite-ancienne-22-LR-Remington-22-Target-item-10653966.html,Remington,Occasion,"11,00 €","4,40 €","Partager\n\n\n Vendre le même objet \nSignaler cet objet\n\n \nDescription produit\n\n\nMunitions tir de loisir, 22LR, Plombs 4,5mm et 9mm Flobert > Munitions - Balles 22LR\n\n\nMarque : RemingtonEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb \n\n\n\n\n\nBoîte en parfait état.\nEnvoie par mondial relais 4€40"


In [4]:
# change dtypes of columns for easier manipulation
df['product_name'] = df['product_name'].astype(str)
df['manufacturer'] = df['manufacturer'].astype(str)
df['is_new'] = df['is_new'].astype(str)
df['price'] = df['price'].astype(str)
df['shipping_cost'] = df['shipping_cost'].astype(str)

# change formatting of prices, remove currency, set as float
df['price'] = df['price'].str.replace(',', '.').str.extract('(\d+\.\d+)', expand=False).astype(float)
df['shipping_cost'] = df['shipping_cost'].str.replace(',', '.').str.extract('(\d+\.\d+)', expand=False).fillna(0).astype(float)

# change string values for new-used to binary
df["is_new"] = df["is_new"].map({"Neuf": 1, "Occasion": 0})
# change missing values to 0 (used item)
df["is_new"] = df["is_new"].fillna(0).astype(int)

# add new column for Total price
df['total_price'] = df['price'] + df['shipping_cost']

In [5]:
def clean_text(text):
    # replace newline characters and extra spaces
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    # remove leading and trailing spaces
    text = text.strip()
    return text

In [6]:
# apply clean_text() function to the 'product_description' column
df['product_description'] = df['product_description'].apply(clean_text)

# remove text from description that doesnt belong to the item itself, eg share buttons and shop category
df['product_description'] = df['product_description'].apply(lambda x: x.split("Flobert > Munitions - Balles 22LR")[1].strip())

In [7]:
df.to_csv('scraped_data_clean.csv', index=False)

In [8]:
# bullet qtty extraction

In [9]:
# boxes of ammo qtty extraction

In [10]:
df.head()

,product_name,product_link,manufacturer,is_new,price,shipping_cost,product_description,total_price
0,BOITE DE 325 CARTOUCHES 22LR FEDERAL GOLD PLATED HOLLOW POINT 36 GRAINS,https://www.naturabuy.fr/BOITE-DE-325-CARTOUCHES-22LR-FEDERAL-GOLD-PLATED-HOLLOW-POINT-36-GRAINS-item-10619103.html,Federal,1,46.9,12.0,Marque : FederalEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Creuse boite de 325 cartouches de marque Federal 22lr 36 grains round nose hollow point cuivree excellente cartouche pour le tir dans les armes a verrou comme semi automatique marque : Fédéral référence : 725 Type d’ogive round nose cuivree a pointe creuse vitesse 384 mètres par secondes poids ogive : 36 grains en stock envoi des reception du paiement vente uniquement aux personnes ayant une licence de tir ou permis de chasse avec validation,58.9
1,Boîte ancienne 22 LR Remington 22 Target,https://www.naturabuy.fr/Boite-ancienne-22-LR-Remington-22-Target-item-10653966.html,Remington,0,11.0,4.4,Marque : RemingtonEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte en parfait état. Envoie par mondial relais 4€40,15.4
2,"boite de 325 cartouches de 22Lr de marque Federal , Automatch , pour semi auto",https://www.naturabuy.fr/boite-325-cartouches-22Lr-marque-Federal-Automatch-semi-auto-item-10619099.html,Federal,1,44.9,12.0,"Marque : FederalEtat de l'objet : NeufType : Standard Longue - haute vitesseType d'ogive : Plomb 1 boite de 325 cartouches de marque Federal 22lr ,40 grains round nose special semi auto Type d'ogive round nose en plomb vitesse 360 metres par secondes poids ogive : 40 grains en stock envoi des reception du paiement vente uniquement aux personnes ayant une licence de tir ou permis de chasse avec validation",56.9
3,Boîte ancienne 22 LR Fiocchi expansive,https://www.naturabuy.fr/Boite-ancienne-22-LR-Fiocchi-expansive-item-10653962.html,Fiocchi,0,12.0,4.4,"Marque : FiocchiEtat de l'objet : D'occasionType : StandardType d'ogive : Plomb Boîte ancienne, Fiocchi expansive, il manque 6 cartouches. Envoie par mondial relais 4,40 €",16.4
4,Boîte ancienne 22 LR NCS 20 Match,https://www.naturabuy.fr/Boite-ancienne-22-LR-NCS-20-Match-item-10653952.html,NCS,0,20.0,4.4,"Marque : NCSEtat de l'objet : D'occasionType : MatchType d'ogive : Plomb Boîte ancienne frabriquée par la Nouvelle Cartoucherie de Survilliers (Cartoucherie Française). Boîte en parfait état. Envoie par mondial relay 4,40 €",24.4


In [11]:
# check for amount of missing values
df.isna().sum()

product_name           0
product_link           0
manufacturer           0
is_new                 0
price                  0
shipping_cost          0
product_description    0
total_price            0
dtype: int64